<a href="https://colab.research.google.com/github/ainizamova/ASR_CourseWork/blob/main/en_speech_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Импортирование и установки**

In [ ]:
# Установка зависимостей
!pip install wget
!apt-get install sox libsndfile1 ffmpeg libsox-fmt-mp3
!pip install unidecode
!pip install matplotlib>=3.3.2
#!pip install --upgrade numba

# Установка NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
# exit()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
libsox-fmt-mp3 is already the newest version (14.4.2-3ubuntu0.18.04.1).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-orxngets/nemo-toolkit_e7d7168e957448578ff669b049f428ec
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-orxngets/nemo-toolkit_e7d7168e957448578ff669b049f428ec
  Using cached setuptools-59.5.0-py3-none-any.whl (952 kB)
  Using cached matplotlib-3.5.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
     |████████████████████████████████| 900 kB 5.3 MB/s 
  Attempting uninstall: setuptools
    F

'\nRemember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!\nAlternatively, you can uncomment the exit() below to crash and restart the kernel, in the case\nthat you want to use the "Run All Cells" (or similar) option.\n'

In [ ]:
# Импорт NeMo
import nemo
# Импорт коллекции по распознаванию речи
import nemo.collections.asr as nemo_asr
# для прослушивания аудио
import IPython
from nemo.collections.asr.metrics.wer import WER, word_error_rate

#from omegaconf import DictConfig, OmegaConf
#from nemo.utils.exp_manager import exp_manager

[NeMo W 2022-04-17 10:43:49 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
# необходимо для RNNT моделей
pip install --upgrade numba --ignore-installed

     |████████████████████████████████| 3.3 MB 5.2 MB/s 
     |████████████████████████████████| 34.5 MB 11 kB/s 
     |████████████████████████████████| 1.1 MB 41.0 MB/s 
     |████████████████████████████████| 15.7 MB 30.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
nemo-toolkit 1.9.0rc0 requires setuptools==59.5.0, but you have setuptools 62.1.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
asr_models = [model for model in dir(nemo_asr.models) if model.endswith("Model")]
asr_models

['ASRModel',
 'EncDecCTCModel',
 'EncDecClassificationModel',
 'EncDecK2SeqModel',
 'EncDecRNNTBPEModel',
 'EncDecRNNTModel',
 'EncDecSpeakerLabelModel']

# **Подготовка данных**

In [ ]:
!pwd

/content


In [ ]:
# Будем располагать данные в текущей директории
data_dir = '.'

In [ ]:
import glob
import os
import subprocess
import tarfile
import wget

In [ ]:
# Если что-то пойдет не так во время обработки данных, раскомментируйте следующую строку, чтобы удалить кэшированный набор данных 
#!rm -rf datasets/...
!mkdir -p datasets/my_en_audio

## Загрузим тестовую часть датасета LibriSpeech

In [ ]:
!mkdir -p scripts
if not os.path.exists("scripts/get_librispeech_data.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/get_librispeech_data.py

In [ ]:
!python scripts/get_librispeech_data.py \
  --data_root "datasets/" \
  --data_set=test_clean,test_other 

# 4 минуты

100% 87/87 [00:14<00:00,  6.20it/s]
100% 90/90 [00:14<00:00,  6.10it/s]


In [ ]:
test_clean_manifest = data_dir + '/datasets/test_clean.json'
test_other_manifest = data_dir + '/datasets/test_other.json'

In [ ]:
def get_audio_path(manifest_path, wav_path):
  audio_paths = []
  with open(manifest_path, 'r') as fin:
    for line  in fin:
      file_id = line[line.rfind('/')+1 : line.find('.wav')]  # напр., "8230-279154-0043"
      audio_paths.append(os.path.join(
                    data_dir, wav_path,
                    file_id + '.wav'))
  return (audio_paths)

In [ ]:
test_clean_files = get_audio_path(test_clean_manifest, 'datasets/LibriSpeech/test-clean-processed')
test_other_files = get_audio_path(test_other_manifest, 'datasets/LibriSpeech/test-other-processed')

In [ ]:
# функция для получения текста-эталона
def get_ground_truth(manifest_path):
  ground_truth_lines = []
  with open(manifest_path, 'r') as fin:
    for line  in fin:
      text = line[line.find('"text": "')+9 : line.rfind('"')]  # "только текст" без кавычек
      ground_truth_lines.append(text)
  return (ground_truth_lines)

# **Тестирование различных моделей**

In [ ]:
!mkdir -p scripts
if not os.path.exists("scripts/transcribe_speech.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/transcribe_speech.py
if not os.path.exists("scripts/speech_to_text_eval.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/speech_to_text_eval.py

In [ ]:
test_clean_manifest

'./datasets/test_clean.json'

## Jasper

In [ ]:
jasper_en = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_jasper10x5dr")

[NeMo I 2022-04-16 11:09:12 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_jasper10x5dr/versions/1.0.0rc1/files/stt_en_jasper10x5dr.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo
[NeMo I 2022-04-16 11:09:49 common:705] Instantiating model from pre-trained checkpoint


[NeMo W 2022-04-16 11:10:01 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2022-04-16 11:10:01 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /data2/vo

[NeMo I 2022-04-16 11:10:01 features:259] PADDING: 16
[NeMo I 2022-04-16 11:10:01 features:276] STFT using torch
[NeMo I 2022-04-16 11:10:09 save_restore_connector:158] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo.


In [ ]:
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/jasper/jasper_10x5dr.yaml

--2022-04-16 11:10:35--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/jasper/jasper_10x5dr.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4814 (4.7K) [text/plain]
Saving to: ‘configs/jasper_10x5dr.yaml’

jasper_10x5dr.yaml  100%[===================>]   4.70K  --.-KB/s    in 0s      

2022-04-16 11:10:35 (69.5 MB/s) - ‘configs/jasper_10x5dr.yaml’ saved [4814/4814]



In [ ]:
# --- JASPER Config Information ---#
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel.yaml import YAML
config_path = './configs/jasper_10x5dr.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    jasper_params = yaml.load(f)
print(f'Jasper parameters:', jasper_params)

Jasper parameters: {'name': 'Jasper10x5', 'model': {'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'tarred_shard_strategy': 'scatter', 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}, 'preprocessor': {'_target_': 'nemo.collect

In [ ]:
jasper_output_manifest_clean="jasper_output_manifest_clean.json"
jasper_output_manifest_other="jasper_output_manifest_other.json"
jasper_model="stt_en_jasper10x5dr" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$jasper_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$jasper_output_manifest_clean


[NeMo I 2022-04-16 11:14:28 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_jasper10x5dr
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: jasper_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        language_model

[NeMo W 2022-04-16 11:14:41 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2022-04-16 11:14:41 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /data2/vo

[NeMo I 2022-04-16 11:14:41 features:259] PADDING: 16
[NeMo I 2022-04-16 11:14:41 features:276] STFT using torch
[NeMo I 2022-04-16 11:14:51 save_restore_connector:158] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo.


[NeMo W 2022-04-16 11:14:51 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 11:14:51 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 11:14:51 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 11:14:51 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 11:23:38 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 11:23:38 transcribe_speech:226] Writing transcriptions into file: jasper_output_manifest_clean.json
[NeMo I 2022-04-16 11:23:38 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./jasper_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 11:23:54 speech_to_text_eval:145] Got WER of 0.036043061472915396


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$jasper_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$jasper_output_manifest_other


[NeMo I 2022-04-16 11:24:16 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_jasper10x5dr
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: jasper_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        language_model

[NeMo W 2022-04-16 11:24:28 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2022-04-16 11:24:28 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /data2/vo

[NeMo I 2022-04-16 11:24:28 features:259] PADDING: 16
[NeMo I 2022-04-16 11:24:28 features:276] STFT using torch
[NeMo I 2022-04-16 11:24:36 save_restore_connector:158] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo.


[NeMo W 2022-04-16 11:24:37 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 11:24:37 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 11:24:37 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 11:24:37 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 11:33:05 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 11:33:05 transcribe_speech:226] Writing transcriptions into file: jasper_output_manifest_other.json
[NeMo I 2022-04-16 11:33:05 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./jasper_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 11:33:18 speech_to_text_eval:145] Got WER of 0.1022104197313872


## QuartzNet

In [ ]:
quartznet_en = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2022-04-16 09:37:47 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2022-04-16 09:37:51 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-16 09:37:52 features:259] PADDING: 16
[NeMo I 2022-04-16 09:37:52 features:276] STFT using torch
[NeMo I 2022-04-16 09:38:04 save_restore_connector:158] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


In [ ]:
#for test_clean

for fname, transcription, ground_truth in zip (test_clean_files, quartznet_en.transcribe(paths2audio_files=test_clean_files), get_ground_truth(test_clean_manifest)):
  print(f"Audio in {fname} was recognized as: {transcription}. It must be: {ground_truth}.")

Transcribing:   0%|          | 0/655 [00:00<?, ?it/s]

Audio in ./datasets/LibriSpeech/test-clean-processed/2300-131720-0000.wav was recognized as: the paris plant like that of the crystal palace was a temporary exhibit. It must be: the paris plant like that at the crystal palace was a temporary exhibit.
Audio in ./datasets/LibriSpeech/test-clean-processed/2300-131720-0001.wav was recognized as: the london plant was less temporary but not permanent supplying before it was torn out no fewer than three thousand lamps in hotels churches stores and dwellings in the vicinity of holborn viaduct. It must be: the london plant was less temporary but not permanent supplying before it was torn out no fewer than three thousand lamps in hotels churches stores and dwellings in the vicinity of holborn viaduct.
Audio in ./datasets/LibriSpeech/test-clean-processed/2300-131720-0002.wav was recognized as: there messrs johnson and hammer put into practice many of the ideas now standard in the art and secured much useful data for the work in new york of which 

In [ ]:
# for test_other

for fname, transcription, ground_truth in zip (test_other_files, quartznet_en.transcribe(paths2audio_files=test_other_files), get_ground_truth(test_other_manifest)):
  print(f"Audio in {fname} was recognized as: {transcription}. It must be: {ground_truth}.")

Transcribing:   0%|          | 0/735 [00:00<?, ?it/s]

Audio in ./datasets/LibriSpeech/test-other-processed/533-131564-0000.wav was recognized as: thain hope i fear. It must be: vain hope i fear.
Audio in ./datasets/LibriSpeech/test-other-processed/533-131564-0001.wav was recognized as: mister and missus hawtersley hav been seeing it to grove a fortnight and as missus hargraey still absent and the weather was remarkably fine i never passed the day without seeing my two friends millcend and esser either there or here. It must be: mister and missus hattersley have been staying at the grove a fortnight and as mister hargrave is still absent and the weather was remarkably fine i never passed a day without seeing my two friends milicent and esther either there or here.
Audio in ./datasets/LibriSpeech/test-other-processed/533-131564-0002.wav was recognized as: no unless he can tell you when to expect him home. It must be: no unless you can tell me when to expect him home.
Audio in ./datasets/LibriSpeech/test-other-processed/533-131564-0003.wav w

In [ ]:
quartznet_output_manifest_clean="quartznet_output_manifest_clean.json"
quartznet_output_manifest_other="quartznet_output_manifest_other.json"
quartznet_model="QuartzNet15x5Base-En" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$quartznet_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$quartznet_output_manifest_clean

[NeMo I 2022-04-16 10:54:50 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: QuartzNet15x5Base-En
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: quartznet_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        language_m

[NeMo W 2022-04-16 10:54:52 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 10:54:52 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 10:54:52 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 10:54:52 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 10:55:36 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 10:55:36 transcribe_speech:226] Writing transcriptions into file: quartznet_output_manifest_clean.json
[NeMo I 2022-04-16 10:55:36 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./quartznet_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 10:55:43 speech_to_text_eval:145] Got WER of 0.03826841144248326


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$quartznet_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$quartznet_output_manifest_other


[NeMo I 2022-04-16 10:57:52 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: QuartzNet15x5Base-En
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: quartznet_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        language_m

[NeMo W 2022-04-16 10:57:53 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 10:57:53 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 10:57:53 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 10:57:53 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 10:58:49 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 10:58:49 transcribe_speech:226] Writing transcriptions into file: quartznet_output_manifest_other.json
[NeMo I 2022-04-16 10:58:49 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./quartznet_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 10:59:30 speech_to_text_eval:145] Got WER of 0.100586515866496


## Citrinet 256

In [ ]:
citrinet = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_citrinet_256")

[NeMo I 2022-04-16 12:01:54 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_256/versions/1.0.0rc1/files/stt_en_citrinet_256.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_256/91a9cc5850784b2065e8a0aa3d526fd9/stt_en_citrinet_256.nemo
[NeMo I 2022-04-16 12:01:58 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-16 12:01:59 mixins:167] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-04-16 12:01:59 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:01:59 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:01:59 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a va

[NeMo I 2022-04-16 12:01:59 features:259] PADDING: 16
[NeMo I 2022-04-16 12:01:59 features:276] STFT using torch
[NeMo I 2022-04-16 12:02:01 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_256/91a9cc5850784b2065e8a0aa3d526fd9/stt_en_citrinet_256.nemo.


In [ ]:
citrinet_256_output_manifest_clean="citrinet_256_output_manifest_clean.json"
citrinet_256_output_manifest_other="citrinet_256_output_manifest_other.json"
citrinet_256_model="stt_en_citrinet_256" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_256_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_256_output_manifest_clean


[NeMo I 2022-04-16 12:02:10 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_256
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: citrinet_256_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        language

[NeMo W 2022-04-16 12:02:12 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:02:12 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:02:12 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a va

[NeMo I 2022-04-16 12:02:12 features:259] PADDING: 16
[NeMo I 2022-04-16 12:02:12 features:276] STFT using torch
[NeMo I 2022-04-16 12:02:13 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_256/91a9cc5850784b2065e8a0aa3d526fd9/stt_en_citrinet_256.nemo.


[NeMo W 2022-04-16 12:02:13 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:02:13 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:02:13 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 12:02:13 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:02:36 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 12:02:36 transcribe_speech:226] Writing transcriptions into file: citrinet_256_output_manifest_clean.json
[NeMo I 2022-04-16 12:02:36 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_256_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 12:02:45 speech_to_text_eval:145] Got WER of 0.042776171637248936


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_256_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_256_output_manifest_other


[NeMo I 2022-04-16 12:02:54 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_256
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: citrinet_256_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        language

[NeMo W 2022-04-16 12:02:56 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:02:56 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:02:56 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a va

[NeMo I 2022-04-16 12:02:56 features:259] PADDING: 16
[NeMo I 2022-04-16 12:02:56 features:276] STFT using torch
[NeMo I 2022-04-16 12:02:57 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_256/91a9cc5850784b2065e8a0aa3d526fd9/stt_en_citrinet_256.nemo.


[NeMo W 2022-04-16 12:02:57 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:02:57 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:02:57 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 12:02:57 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:03:21 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 12:03:21 transcribe_speech:226] Writing transcriptions into file: citrinet_256_output_manifest_other.json
[NeMo I 2022-04-16 12:03:21 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_256_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 12:03:26 speech_to_text_eval:145] Got WER of 0.10182832470435398


## Citrinet 512

In [ ]:
citrinet = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_citrinet_512")

[NeMo I 2022-04-16 11:34:40 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_512/versions/1.0.0rc1/files/stt_en_citrinet_512.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_512/3262321355385bb7cf5a583146117d77/stt_en_citrinet_512.nemo
[NeMo I 2022-04-16 11:34:46 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-16 11:34:48 mixins:167] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-04-16 11:34:49 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2022-04-16 11:34:49 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    
[NeMo W 2022-04-16 11:34:49 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    T

[NeMo I 2022-04-16 11:34:49 features:259] PADDING: 16
[NeMo I 2022-04-16 11:34:49 features:276] STFT using torch
[NeMo I 2022-04-16 11:34:51 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_512/3262321355385bb7cf5a583146117d77/stt_en_citrinet_512.nemo.


In [ ]:
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/citrinet/citrinet_512.yaml

--2022-04-16 11:34:51--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/citrinet/citrinet_512.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14307 (14K) [text/plain]
Saving to: ‘configs/citrinet_512.yaml’

citrinet_512.yaml   100%[===================>]  13.97K  --.-KB/s    in 0s      

2022-04-16 11:34:51 (119 MB/s) - ‘configs/citrinet_512.yaml’ saved [14307/14307]



In [ ]:
# --- Citrinet 512 Config Information ---#
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel.yaml import YAML
config_path = './configs/citrinet_512.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    citrinet_params = yaml.load(f)
print(f'Citrinet parameters:', citrinet_params)

Citrinet parameters: {'name': 'Citrinet-512-8x-Stride', 'model': {'sample_rate': 16000, 'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'batch_size': 32, 'trim_silence': False, 'max_duration': 16.7, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False}, 'test_ds': {'manifest_filepath': None, 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False}, 'model_defaults': {'repeat': 5, 'dropout': 0.1, 'separable': True, 'se': True, 'se_context_size': -1, 'kernel_size_factor': 1.0, 'enc_final': 640}, 'tokenizer': {'dir': '???', 'type': '???'}, 'preprocessor': {'_target

In [ ]:
citrinet_512_output_manifest_clean="citrinet_512_output_manifest_clean.json"
citrinet_512_output_manifest_other="citrinet_512_output_manifest_other.json"
citrinet_512_model="stt_en_citrinet_512" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_512_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_512_output_manifest_clean


[NeMo I 2022-04-16 11:45:55 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_512
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: citrinet_512_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        language

[NeMo W 2022-04-16 11:45:58 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2022-04-16 11:45:58 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    
[NeMo W 2022-04-16 11:45:58 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    T

[NeMo I 2022-04-16 11:45:58 features:259] PADDING: 16
[NeMo I 2022-04-16 11:45:58 features:276] STFT using torch
[NeMo I 2022-04-16 11:45:59 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_512/3262321355385bb7cf5a583146117d77/stt_en_citrinet_512.nemo.


[NeMo W 2022-04-16 11:45:59 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 11:45:59 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 11:45:59 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 11:45:59 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 11:46:54 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 11:46:54 transcribe_speech:226] Writing transcriptions into file: citrinet_512_output_manifest_clean.json
[NeMo I 2022-04-16 11:46:54 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_512_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 11:46:57 speech_to_text_eval:145] Got WER of 0.03925745587340231


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_512_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_512_output_manifest_other


[NeMo I 2022-04-16 11:47:15 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_512
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: citrinet_512_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        language

[NeMo W 2022-04-16 11:47:18 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2022-04-16 11:47:18 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    
[NeMo W 2022-04-16 11:47:18 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    T

[NeMo I 2022-04-16 11:47:18 features:259] PADDING: 16
[NeMo I 2022-04-16 11:47:18 features:276] STFT using torch
[NeMo I 2022-04-16 11:47:20 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_512/3262321355385bb7cf5a583146117d77/stt_en_citrinet_512.nemo.


[NeMo W 2022-04-16 11:47:20 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 11:47:20 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 11:47:20 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 11:47:20 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 11:48:14 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 11:48:14 transcribe_speech:226] Writing transcriptions into file: citrinet_512_output_manifest_other.json
[NeMo I 2022-04-16 11:48:14 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_512_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 11:48:25 speech_to_text_eval:145] Got WER of 0.09000248361767571


## Citrinet 1024

In [ ]:
citrinet = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_citrinet_1024")

[NeMo I 2022-04-16 12:05:29 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_1024/versions/1.0.0rc1/files/stt_en_citrinet_1024.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_1024/86acfaf495a53383369fb6c9c547b8dd/stt_en_citrinet_1024.nemo
[NeMo I 2022-04-16 12:05:46 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-16 12:05:52 mixins:167] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-04-16 12:05:53 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:05:53 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:05:53 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a va

[NeMo I 2022-04-16 12:05:53 features:259] PADDING: 16
[NeMo I 2022-04-16 12:05:53 features:276] STFT using torch
[NeMo I 2022-04-16 12:05:58 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_1024/86acfaf495a53383369fb6c9c547b8dd/stt_en_citrinet_1024.nemo.


In [ ]:
citrinet_1024_output_manifest_clean="citrinet_1024_output_manifest_clean.json"
citrinet_1024_output_manifest_other="citrinet_1024_output_manifest_other.json"
citrinet_1024_model="stt_en_citrinet_1024" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_1024_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_1024_output_manifest_clean


[NeMo I 2022-04-16 12:06:11 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_1024
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: citrinet_1024_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        langua

[NeMo W 2022-04-16 12:06:17 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:06:17 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:06:17 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a va

[NeMo I 2022-04-16 12:06:17 features:259] PADDING: 16
[NeMo I 2022-04-16 12:06:17 features:276] STFT using torch
[NeMo I 2022-04-16 12:06:22 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_1024/86acfaf495a53383369fb6c9c547b8dd/stt_en_citrinet_1024.nemo.


[NeMo W 2022-04-16 12:06:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:06:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:06:22 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 12:06:22 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:08:34 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 12:08:34 transcribe_speech:226] Writing transcriptions into file: citrinet_1024_output_manifest_clean.json
[NeMo I 2022-04-16 12:08:34 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_1024_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 12:08:44 speech_to_text_eval:145] Got WER of 0.03138314059646987


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_1024_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_1024_output_manifest_other


[NeMo I 2022-04-16 12:09:06 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_1024
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: citrinet_1024_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        langua

[NeMo W 2022-04-16 12:09:12 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:09:12 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:09:12 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a va

[NeMo I 2022-04-16 12:09:12 features:259] PADDING: 16
[NeMo I 2022-04-16 12:09:12 features:276] STFT using torch
[NeMo I 2022-04-16 12:09:16 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_1024/86acfaf495a53383369fb6c9c547b8dd/stt_en_citrinet_1024.nemo.


[NeMo W 2022-04-16 12:09:16 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:09:16 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:09:16 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 12:09:16 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:11:31 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 12:11:31 transcribe_speech:226] Writing transcriptions into file: citrinet_1024_output_manifest_other.json
[NeMo I 2022-04-16 12:11:32 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_1024_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 12:11:39 speech_to_text_eval:145] Got WER of 0.07045832298492635


## Citrinet 256 gamma 0.25

In [ ]:
citrinet = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_citrinet_256_gamma_0_25")

[NeMo I 2022-04-16 11:56:47 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_256_gamma_0_25/versions/1.0.0/files/stt_en_citrinet_256_gamma_0_25.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_256_gamma_0_25/d6eff3868f2f7a4791eb935c8366fc46/stt_en_citrinet_256_gamma_0_25.nemo
[NeMo I 2022-04-16 11:56:51 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-16 11:56:52 mixins:167] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-04-16 11:56:53 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:56:53 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:56:53 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a v

[NeMo I 2022-04-16 11:56:53 features:259] PADDING: 16
[NeMo I 2022-04-16 11:56:53 features:276] STFT using torch
[NeMo I 2022-04-16 11:56:54 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_256_gamma_0_25/d6eff3868f2f7a4791eb935c8366fc46/stt_en_citrinet_256_gamma_0_25.nemo.


In [ ]:
citrinet_256_gamma_output_manifest_clean="citrinet_256_gamma_output_manifest_clean.json"
citrinet_256_gamma_output_manifest_other="citrinet_256_gamma_output_manifest_other.json"
citrinet_256_gamma_model="stt_en_citrinet_256_gamma_0_25" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_256_gamma_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_256_gamma_output_manifest_clean


[NeMo I 2022-04-16 11:57:36 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_256_gamma_0_25
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: citrinet_256_gamma_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2

[NeMo W 2022-04-16 11:57:38 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:57:38 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:57:38 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a v

[NeMo I 2022-04-16 11:57:38 features:259] PADDING: 16
[NeMo I 2022-04-16 11:57:38 features:276] STFT using torch
[NeMo I 2022-04-16 11:57:39 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_256_gamma_0_25/d6eff3868f2f7a4791eb935c8366fc46/stt_en_citrinet_256_gamma_0_25.nemo.


[NeMo W 2022-04-16 11:57:39 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 11:57:39 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 11:57:39 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 11:57:39 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 11:58:07 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 11:58:07 transcribe_speech:226] Writing transcriptions into file: citrinet_256_gamma_output_manifest_clean.json
[NeMo I 2022-04-16 11:58:07 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_256_gamma_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 11:58:10 speech_to_text_eval:145] Got WER of 0.047150791235544734


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_256_gamma_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_256_gamma_output_manifest_other


[NeMo I 2022-04-16 11:58:29 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_256_gamma_0_25
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: citrinet_256_gamma_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2

[NeMo W 2022-04-16 11:58:31 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:58:31 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:58:31 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a v

[NeMo I 2022-04-16 11:58:31 features:259] PADDING: 16
[NeMo I 2022-04-16 11:58:31 features:276] STFT using torch
[NeMo I 2022-04-16 11:58:32 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_256_gamma_0_25/d6eff3868f2f7a4791eb935c8366fc46/stt_en_citrinet_256_gamma_0_25.nemo.


[NeMo W 2022-04-16 11:58:32 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 11:58:32 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 11:58:32 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 11:58:32 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 11:58:59 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 11:58:59 transcribe_speech:226] Writing transcriptions into file: citrinet_256_gamma_output_manifest_other.json
[NeMo I 2022-04-16 11:58:59 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_256_gamma_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 11:59:39 speech_to_text_eval:145] Got WER of 0.10633704602334601


## Citrinet 512 gamma 0.25

In [ ]:
citrinet = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_citrinet_512_gamma_0_25")

[NeMo I 2022-04-16 11:50:24 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_512_gamma_0_25/versions/1.0.0/files/stt_en_citrinet_512_gamma_0_25.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_512_gamma_0_25/1497f738f22b48b92849481b7b5100e4/stt_en_citrinet_512_gamma_0_25.nemo
[NeMo I 2022-04-16 11:50:31 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-16 11:50:33 mixins:167] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-04-16 11:50:34 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:50:34 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:50:34 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a v

[NeMo I 2022-04-16 11:50:34 features:259] PADDING: 16
[NeMo I 2022-04-16 11:50:34 features:276] STFT using torch
[NeMo I 2022-04-16 11:50:36 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_512_gamma_0_25/1497f738f22b48b92849481b7b5100e4/stt_en_citrinet_512_gamma_0_25.nemo.


In [ ]:
citrinet_512_gamma_output_manifest_clean="citrinet_512_gamma_output_manifest_clean.json"
citrinet_512_gamma_output_manifest_other="citrinet_512_gamma_output_manifest_other.json"
citrinet_512_gamma_model="stt_en_citrinet_512_gamma_0_25" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_512_gamma_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_512_gamma_output_manifest_clean


[NeMo I 2022-04-16 11:52:40 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_512_gamma_0_25
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: citrinet_512_gamma_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2

[NeMo W 2022-04-16 11:52:42 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:52:42 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:52:42 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a v

[NeMo I 2022-04-16 11:52:42 features:259] PADDING: 16
[NeMo I 2022-04-16 11:52:42 features:276] STFT using torch
[NeMo I 2022-04-16 11:52:44 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_512_gamma_0_25/1497f738f22b48b92849481b7b5100e4/stt_en_citrinet_512_gamma_0_25.nemo.


[NeMo W 2022-04-16 11:52:44 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 11:52:44 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 11:52:44 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 11:52:44 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 11:53:21 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 11:53:21 transcribe_speech:226] Writing transcriptions into file: citrinet_512_gamma_output_manifest_clean.json
[NeMo I 2022-04-16 11:53:21 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_512_gamma_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 11:53:48 speech_to_text_eval:145] Got WER of 0.038705873402312845


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_512_gamma_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_512_gamma_output_manifest_other


[NeMo I 2022-04-16 11:53:58 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_512_gamma_0_25
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: citrinet_512_gamma_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2

[NeMo W 2022-04-16 11:54:00 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:54:00 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 11:54:00 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a v

[NeMo I 2022-04-16 11:54:00 features:259] PADDING: 16
[NeMo I 2022-04-16 11:54:00 features:276] STFT using torch
[NeMo I 2022-04-16 11:54:02 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_512_gamma_0_25/1497f738f22b48b92849481b7b5100e4/stt_en_citrinet_512_gamma_0_25.nemo.


[NeMo W 2022-04-16 11:54:02 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 11:54:02 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 11:54:02 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 11:54:02 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 11:54:39 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 11:54:39 transcribe_speech:226] Writing transcriptions into file: citrinet_512_gamma_output_manifest_other.json
[NeMo I 2022-04-16 11:54:39 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_512_gamma_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 11:54:41 speech_to_text_eval:145] Got WER of 0.08990695986091742


## Citrinet 1024 gamma 0.25

In [ ]:
citrinet = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_citrinet_1024_gamma_0_25")

[NeMo I 2022-04-16 12:11:50 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_1024_gamma_0_25/versions/1.0.0/files/stt_en_citrinet_1024_gamma_0_25.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_1024_gamma_0_25/a31810fe7dd91146ddb79361b5cc8907/stt_en_citrinet_1024_gamma_0_25.nemo
[NeMo I 2022-04-16 12:12:06 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-16 12:12:12 mixins:167] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-04-16 12:12:13 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:12:13 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:12:13 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a v

[NeMo I 2022-04-16 12:12:13 features:259] PADDING: 16
[NeMo I 2022-04-16 12:12:13 features:276] STFT using torch
[NeMo I 2022-04-16 12:12:17 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_1024_gamma_0_25/a31810fe7dd91146ddb79361b5cc8907/stt_en_citrinet_1024_gamma_0_25.nemo.


In [ ]:
citrinet_1024_gamma_output_manifest_clean="citrinet_1024_gamma_output_manifest_clean.json"
citrinet_1024_gamma_output_manifest_other="citrinet_1024_gamma_output_manifest_other.json"
citrinet_1024_gamma_model="stt_en_citrinet_1024_gamma_0_25" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_1024_gamma_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_1024_gamma_output_manifest_clean


[NeMo I 2022-04-16 12:12:37 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_1024_gamma_0_25
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: citrinet_1024_gamma_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta:

[NeMo W 2022-04-16 12:12:43 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:12:43 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:12:43 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a v

[NeMo I 2022-04-16 12:12:43 features:259] PADDING: 16
[NeMo I 2022-04-16 12:12:43 features:276] STFT using torch
[NeMo I 2022-04-16 12:12:47 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_1024_gamma_0_25/a31810fe7dd91146ddb79361b5cc8907/stt_en_citrinet_1024_gamma_0_25.nemo.


[NeMo W 2022-04-16 12:12:47 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:12:47 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:12:48 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 12:12:48 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:14:05 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 12:14:05 transcribe_speech:226] Writing transcriptions into file: citrinet_1024_gamma_output_manifest_clean.json
[NeMo I 2022-04-16 12:14:05 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_1024_gamma_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 12:14:40 speech_to_text_eval:145] Got WER of 0.03370359099208765


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$citrinet_1024_gamma_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$citrinet_1024_gamma_output_manifest_other


[NeMo I 2022-04-16 12:14:50 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_citrinet_1024_gamma_0_25
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: citrinet_1024_gamma_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta:

[NeMo W 2022-04-16 12:14:56 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:14:56 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-16 12:14:56 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a v

[NeMo I 2022-04-16 12:14:56 features:259] PADDING: 16
[NeMo I 2022-04-16 12:14:56 features:276] STFT using torch
[NeMo I 2022-04-16 12:15:00 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_citrinet_1024_gamma_0_25/a31810fe7dd91146ddb79361b5cc8907/stt_en_citrinet_1024_gamma_0_25.nemo.


[NeMo W 2022-04-16 12:15:00 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:15:00 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:15:01 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 12:15:01 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:16:18 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 12:16:18 transcribe_speech:226] Writing transcriptions into file: citrinet_1024_gamma_output_manifest_other.json
[NeMo I 2022-04-16 12:16:19 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./citrinet_1024_gamma_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 12:16:41 speech_to_text_eval:145] Got WER of 0.07412643524444529


## Conformer-CTC small LS

In [ ]:
conformer_ctc_small_ls = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_small_ls")

[NeMo I 2022-04-16 12:50:12 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_small_ls/versions/1.0.0/files/stt_en_conformer_ctc_small_ls.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo
[NeMo I 2022-04-16 12:50:16 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-16 12:50:17 mixins:167] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-04-16 12:50:17 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2022-04-16 12:50:17 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2022-04-16 12:50:17 features:259] PADDING: 0
[NeMo I 2022-04-16 12:50:17 features:276] STFT using torch
[NeMo I 2022-04-16 12:50:18 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo.


In [ ]:
conformer_ctc_small_ls_output_manifest_clean="conformer_ctc_small_ls_output_manifest_clean.json"
conformer_ctc_small_ls_output_manifest_other="conformer_ctc_small_ls_output_manifest_other.json"
conformer_ctc_small_ls_model="stt_en_conformer_ctc_small_ls" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_ctc_small_ls_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_ctc_small_ls_output_manifest_clean


[NeMo I 2022-04-16 12:51:01 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_ctc_small_ls
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: conformer_ctc_small_ls_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta

[NeMo W 2022-04-16 12:51:02 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2022-04-16 12:51:02 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2022-04-16 12:51:02 features:259] PADDING: 0
[NeMo I 2022-04-16 12:51:02 features:276] STFT using torch
[NeMo I 2022-04-16 12:51:02 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo.


[NeMo W 2022-04-16 12:51:02 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:51:02 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:51:02 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 12:51:02 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:52:30 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 12:52:30 transcribe_speech:226] Writing transcriptions into file: conformer_ctc_small_ls_output_manifest_clean.json
[NeMo I 2022-04-16 12:52:30 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_ctc_small_ls_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 12:52:47 speech_to_text_eval:145] Got WER of 0.03492087644552647


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_ctc_small_ls_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_ctc_small_ls_output_manifest_other


[NeMo I 2022-04-16 12:52:59 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_ctc_small_ls
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: conformer_ctc_small_ls_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta

[NeMo W 2022-04-16 12:52:59 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2022-04-16 12:52:59 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2022-04-16 12:52:59 features:259] PADDING: 0
[NeMo I 2022-04-16 12:52:59 features:276] STFT using torch
[NeMo I 2022-04-16 12:53:00 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo.


[NeMo W 2022-04-16 12:53:00 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:53:00 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:53:00 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 12:53:00 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:54:27 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 12:54:27 transcribe_speech:226] Writing transcriptions into file: conformer_ctc_small_ls_output_manifest_other.json
[NeMo I 2022-04-16 12:54:27 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_ctc_small_ls_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 12:55:01 speech_to_text_eval:145] Got WER of 0.08792006572034465


## Conformer-CTC large LS

In [ ]:
conformer_ctc_large_ls = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large_ls")

[NeMo I 2022-04-17 09:40:45 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_large_ls/versions/1.0.0/files/stt_en_conformer_ctc_large_ls.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_large_ls/1ea6a779af1742c38ff9945aa70b3dc3/stt_en_conformer_ctc_large_ls.nemo
[NeMo I 2022-04-17 09:40:58 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-17 09:41:03 mixins:167] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-04-17 09:41:03 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2022-04-17 09:41:03 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2022-04-17 09:41:03 features:259] PADDING: 0
[NeMo I 2022-04-17 09:41:03 features:276] STFT using torch
[NeMo I 2022-04-17 09:41:05 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_large_ls/1ea6a779af1742c38ff9945aa70b3dc3/stt_en_conformer_ctc_large_ls.nemo.


In [ ]:
conformer_ctc_large_ls_output_manifest_clean="conformer_ctc_large_ls_output_manifest_clean.json"
conformer_ctc_large_ls_output_manifest_other="conformer_ctc_large_ls_output_manifest_other.json"
conformer_ctc_large_ls_model="stt_en_conformer_ctc_large_ls" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_ctc_large_ls_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_ctc_large_ls_output_manifest_clean


[NeMo I 2022-04-17 09:41:12 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_ctc_large_ls
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: conformer_ctc_large_ls_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta

[NeMo W 2022-04-17 09:41:17 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2022-04-17 09:41:17 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2022-04-17 09:41:17 features:259] PADDING: 0
[NeMo I 2022-04-17 09:41:17 features:276] STFT using torch
[NeMo I 2022-04-17 09:41:19 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_large_ls/1ea6a779af1742c38ff9945aa70b3dc3/stt_en_conformer_ctc_large_ls.nemo.


[NeMo W 2022-04-17 09:41:19 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-17 09:41:19 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-17 09:41:19 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-17 09:41:19 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-17 10:00:57 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-17 10:00:57 transcribe_speech:226] Writing transcriptions into file: conformer_ctc_large_ls_output_manifest_clean.json
[NeMo I 2022-04-17 10:00:57 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_ctc_large_ls_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-17 10:01:02 speech_to_text_eval:145] Got WER of 0.02775030432136336


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_ctc_large_ls_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_ctc_large_ls_output_manifest_other


[NeMo I 2022-04-17 10:01:14 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_ctc_large_ls
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: conformer_ctc_large_ls_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta

[NeMo W 2022-04-17 10:01:20 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2022-04-17 10:01:20 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2022-04-17 10:01:20 features:259] PADDING: 0
[NeMo I 2022-04-17 10:01:20 features:276] STFT using torch
[NeMo I 2022-04-17 10:01:22 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_large_ls/1ea6a779af1742c38ff9945aa70b3dc3/stt_en_conformer_ctc_large_ls.nemo.


[NeMo W 2022-04-17 10:01:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-17 10:01:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-17 10:01:22 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-17 10:01:22 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-17 10:20:20 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-17 10:20:20 transcribe_speech:226] Writing transcriptions into file: conformer_ctc_large_ls_output_manifest_other.json
[NeMo I 2022-04-17 10:20:20 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_ctc_large_ls_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-17 10:22:31 speech_to_text_eval:145] Got WER of 0.061918499130733815


## Conformer-CTC large

In [ ]:
conformer_ctc_large = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large")

[NeMo I 2022-04-16 12:26:46 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_large/versions/1.6.0/files/stt_en_conformer_ctc_large.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo
[NeMo I 2022-04-16 12:27:01 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-16 12:27:06 mixins:167] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-04-16 12:27:06 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-04-16 12:27:06 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-04-16 12:27:06 features:259] PADDING: 0
[NeMo I 2022-04-16 12:27:06 features:276] STFT using torch
[NeMo I 2022-04-16 12:27:08 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo.


In [ ]:
conformer_ctc_large_output_manifest_clean="conformer_ctc_large_output_manifest_clean.json"
conformer_ctc_large_output_manifest_other="conformer_ctc_large_output_manifest_other.json"
conformer_ctc_large_model="stt_en_conformer_ctc_large" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_ctc_large_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_ctc_large_output_manifest_clean


[NeMo I 2022-04-16 12:27:44 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_ctc_large
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: conformer_ctc_large_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
  

[NeMo W 2022-04-16 12:27:48 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-04-16 12:27:48 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-04-16 12:27:48 features:259] PADDING: 0
[NeMo I 2022-04-16 12:27:48 features:276] STFT using torch
[NeMo I 2022-04-16 12:27:50 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo.


[NeMo W 2022-04-16 12:27:50 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:27:50 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:27:50 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-16 12:27:50 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:33:34 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-16 12:33:34 transcribe_speech:226] Writing transcriptions into file: conformer_ctc_large_output_manifest_clean.json
[NeMo I 2022-04-16 12:33:34 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_ctc_large_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-16 12:33:48 speech_to_text_eval:145] Got WER of 0.02198721850273889


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_ctc_large_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_ctc_large_output_manifest_other


[NeMo I 2022-04-16 12:34:15 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_ctc_large
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: conformer_ctc_large_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
  

[NeMo W 2022-04-16 12:34:20 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-04-16 12:34:20 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-04-16 12:34:20 features:259] PADDING: 0
[NeMo I 2022-04-16 12:34:20 features:276] STFT using torch
[NeMo I 2022-04-16 12:34:21 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo.


[NeMo W 2022-04-16 12:34:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:34:22 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:34:22 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 12:34:22 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:40:00 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 12:40:00 transcribe_speech:226] Writing transcriptions into file: conformer_ctc_large_output_manifest_other.json
[NeMo I 2022-04-16 12:40:00 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_ctc_large_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 12:40:46 speech_to_text_eval:145] Got WER of 0.043520623579084115


## Conformer-CTC small

In [ ]:
conformer_ctc_small = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_small")

[NeMo I 2022-04-17 09:23:58 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_small/versions/1.6.0/files/stt_en_conformer_ctc_small.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo
[NeMo I 2022-04-17 09:24:02 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-17 09:24:03 mixins:167] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-04-17 09:24:04 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-04-17 09:24:04 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-04-17 09:24:04 features:259] PADDING: 0
[NeMo I 2022-04-17 09:24:04 features:276] STFT using torch
[NeMo I 2022-04-17 09:24:17 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.


In [ ]:
conformer_ctc_small_output_manifest_clean="conformer_ctc_small_output_manifest_clean.json"
conformer_ctc_small_output_manifest_other="conformer_ctc_small_output_manifest_other.json"
conformer_ctc_small_model="stt_en_conformer_ctc_small" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_ctc_small_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_ctc_small_output_manifest_clean


[NeMo I 2022-04-17 09:24:17 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_ctc_small
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: conformer_ctc_small_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
  

[NeMo W 2022-04-17 09:24:19 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-04-17 09:24:19 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-04-17 09:24:19 features:259] PADDING: 0
[NeMo I 2022-04-17 09:24:19 features:276] STFT using torch
[NeMo I 2022-04-17 09:24:20 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.


[NeMo W 2022-04-17 09:24:20 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-17 09:24:20 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-17 09:24:20 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-17 09:24:20 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-17 09:28:57 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-17 09:28:57 transcribe_speech:226] Writing transcriptions into file: conformer_ctc_small_output_manifest_clean.json
[NeMo I 2022-04-17 09:28:57 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_ctc_small_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-17 09:29:24 speech_to_text_eval:145] Got WER of 0.03712720632988436


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_ctc_small_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_ctc_small_output_manifest_other


[NeMo I 2022-04-17 09:30:49 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_ctc_small
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: conformer_ctc_small_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
  

[NeMo W 2022-04-17 09:30:51 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-04-17 09:30:51 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-04-17 09:30:51 features:259] PADDING: 0
[NeMo I 2022-04-17 09:30:51 features:276] STFT using torch
[NeMo I 2022-04-17 09:30:52 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.


[NeMo W 2022-04-17 09:30:52 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-17 09:30:52 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-17 09:30:52 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-17 09:30:52 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-17 09:35:25 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-17 09:35:25 transcribe_speech:226] Writing transcriptions into file: conformer_ctc_small_output_manifest_other.json
[NeMo I 2022-04-17 09:35:26 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_ctc_small_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-17 09:35:26 speech_to_text_eval:145] Got WER of 0.08188296429321973


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$contextnet_256_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$contextnet_256_output_manifest_other

[NeMo I 2022-04-16 12:52:59 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_ctc_small_ls
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: conformer_ctc_small_ls_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta

[NeMo W 2022-04-16 12:52:59 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2022-04-16 12:52:59 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2022-04-16 12:52:59 features:259] PADDING: 0
[NeMo I 2022-04-16 12:52:59 features:276] STFT using torch
[NeMo I 2022-04-16 12:53:00 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo.


[NeMo W 2022-04-16 12:53:00 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-16 12:53:00 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-16 12:53:00 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-16 12:53:00 transcribe_speech:176] AMP enabled!
    


Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-16 12:54:27 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-16 12:54:27 transcribe_speech:226] Writing transcriptions into file: conformer_ctc_small_ls_output_manifest_other.json
[NeMo I 2022-04-16 12:54:27 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./contextnet_256_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-16 12:55:01 speech_to_text_eval:145] Got WER of 0.08792006572034465


## Contextnet 256

In [ ]:
contextnet_256 = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="stt_en_contextnet_256")

[NeMo I 2022-04-17 11:03:44 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_contextnet_256/versions/1.6.0/files/stt_en_contextnet_256.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_contextnet_256/bec0c4dd71d3e3cf5dd20458ed251a89/stt_en_contextnet_256.nemo
[NeMo I 2022-04-17 11:03:48 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-17 11:03:51 mixins:167] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-04-17 11:03:52 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    use_start_end_token: false
    
[NeMo W 2022-04-17 11:03:52 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-17 11:03:52 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2022-04-17 11:03:52 features:259] PADDING: 16
[NeMo I 2022-04-17 11:03:52 features:276] STFT using torch


[NeMo W 2022-04-17 11:03:53 rnnt:745] `experimental_fuse_loss_wer` will be deprecated in NeMo 1.6. Please use `fuse_loss_wer` instead.


[NeMo I 2022-04-17 11:03:53 rnnt_models:210] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.001}
[NeMo I 2022-04-17 11:03:53 audio_preprocessing:496] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-04-17 11:03:54 save_restore_connector:158] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_contextnet_256/bec0c4dd71d3e3cf5dd20458ed251a89/stt_en_contextnet_256.nemo.


In [ ]:
contextnet_256.summarize()

[NeMo W 2022-04-17 11:03:54 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: LightningDeprecationWarning: The `LightningModule.summarize` method is deprecated in v1.5 and will be removed in v1.7. Use `pytorch_lightning.utilities.model_summary.summarize` instead.
      exec(code_obj, self.user_global_ns, self.user_ns)
    


  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 9.1 M 
2 | decoder           | RNNTDecoder                       | 3.9 M 
3 | joint             | RNNTJoint                         | 1.5 M 
4 | loss              | RNNTLoss                          | 0     
5 | spec_augmentation | SpectrogramAugmentation           | 0     
6 | wer               | RNNTBPEWER                        | 0     
------------------------------------------------------------------------
14.5 M    Trainable params
0         Non-trainable params
14.5 M    Total params
58.090    Total estimated model params size (MB)

In [ ]:
contextnet_256_output_manifest_clean="contextnet_256_output_manifest_clean.json"
contextnet_256_output_manifest_other="contextnet_256_output_manifest_other.json"
contextnet_256_model="stt_en_contextnet_256" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$contextnet_256_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$contextnet_256_output_manifest_clean


[NeMo I 2022-04-17 11:04:18 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_contextnet_256
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: contextnet_256_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        lang

[NeMo W 2022-04-17 11:04:21 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    use_start_end_token: false
    
[NeMo W 2022-04-17 11:04:21 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-17 11:04:21 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2022-04-17 11:04:21 features:259] PADDING: 16
[NeMo I 2022-04-17 11:04:21 features:276] STFT using torch


[NeMo W 2022-04-17 11:04:22 rnnt:745] `experimental_fuse_loss_wer` will be deprecated in NeMo 1.6. Please use `fuse_loss_wer` instead.


[NeMo I 2022-04-17 11:04:22 rnnt_models:210] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.001}
[NeMo I 2022-04-17 11:04:22 audio_preprocessing:496] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-04-17 11:04:23 save_restore_connector:158] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_contextnet_256/bec0c4dd71d3e3cf5dd20458ed251a89/stt_en_contextnet_256.nemo.


[NeMo W 2022-04-17 11:04:23 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-17 11:04:23 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-17 11:04:23 rnnt_wer:366] Joint fused batch size <= 0; Will temporarily disable fused batch step in the Joint.
[NeMo I 2022-04-17 11:04:23 rnnt_bpe_models:377] Changed decoding strategy to 
    strategy: greedy_batch
    compute_hypothesis_token_set: false
    preserve_alignments: null
    fused_batch_size: -1
    greedy:
      max_symbols_per_step: 10
      preserve_alignments: false
    beam:
      beam_size: 4
      search_type: default
      score_norm: true
      return_best_hypothesis: true
      tsd_max_sym_exp_per_step: 50
      alsd_max_target_len: 1.0
      nsc_max_timesteps_expansion: 1
      nsc_prefix_alpha: 1
      maes_num_steps: 2
      maes_prefix_alpha: 1
      maes_expansion_gamma: 2.3
      maes_expansion_beta: 2
      language_model: null
      softmax_temperature: 1.0
      preserve_alignments: false
    
[NeMo I 2022-04-17 11:04:23 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-17 11:04:23 transcribe_speech:176] AMP en

Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-17 11:07:37 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-17 11:07:37 transcribe_speech:226] Writing transcriptions into file: contextnet_256_output_manifest_clean.json
[NeMo I 2022-04-17 11:07:37 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./contextnet_256_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-17 11:07:39 speech_to_text_eval:145] Got WER of 0.03315200852099817


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$contextnet_256_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$contextnet_256_output_manifest_other


[NeMo I 2022-04-17 11:07:46 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_contextnet_256
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: contextnet_256_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta: 2
        lang

[NeMo W 2022-04-17 11:07:50 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: false
    max_duration: 20.0
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    use_start_end_token: false
    
[NeMo W 2022-04-17 11:07:50 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2022-04-17 11:07:50 modelPT:162] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2022-04-17 11:07:50 features:259] PADDING: 16
[NeMo I 2022-04-17 11:07:50 features:276] STFT using torch


[NeMo W 2022-04-17 11:07:51 rnnt:745] `experimental_fuse_loss_wer` will be deprecated in NeMo 1.6. Please use `fuse_loss_wer` instead.


[NeMo I 2022-04-17 11:07:51 rnnt_models:210] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.001}
[NeMo I 2022-04-17 11:07:51 audio_preprocessing:496] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-04-17 11:07:51 save_restore_connector:158] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_contextnet_256/bec0c4dd71d3e3cf5dd20458ed251a89/stt_en_contextnet_256.nemo.


[NeMo W 2022-04-17 11:07:51 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-17 11:07:51 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-17 11:07:51 rnnt_wer:366] Joint fused batch size <= 0; Will temporarily disable fused batch step in the Joint.
[NeMo I 2022-04-17 11:07:51 rnnt_bpe_models:377] Changed decoding strategy to 
    strategy: greedy_batch
    compute_hypothesis_token_set: false
    preserve_alignments: null
    fused_batch_size: -1
    greedy:
      max_symbols_per_step: 10
      preserve_alignments: false
    beam:
      beam_size: 4
      search_type: default
      score_norm: true
      return_best_hypothesis: true
      tsd_max_sym_exp_per_step: 50
      alsd_max_target_len: 1.0
      nsc_max_timesteps_expansion: 1
      nsc_prefix_alpha: 1
      maes_num_steps: 2
      maes_prefix_alpha: 1
      maes_expansion_gamma: 2.3
      maes_expansion_beta: 2
      language_model: null
      softmax_temperature: 1.0
      preserve_alignments: false
    
[NeMo I 2022-04-17 11:07:51 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-17 11:07:51 transcribe_speech:176] AMP en

Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-17 11:11:04 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-17 11:11:04 transcribe_speech:226] Writing transcriptions into file: contextnet_256_output_manifest_other.json
[NeMo I 2022-04-17 11:11:04 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./contextnet_256_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-17 11:11:08 speech_to_text_eval:145] Got WER of 0.0802208509256252


## Conformer-Transducer small

In [ ]:
conformer_tr_small = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="stt_en_conformer_transducer_small")

[NeMo I 2022-04-17 10:46:36 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_transducer_small/a755afe69952642a8410330876938b83/stt_en_conformer_transducer_small.nemo.
[NeMo I 2022-04-17 10:46:36 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_transducer_small/a755afe69952642a8410330876938b83/stt_en_conformer_transducer_small.nemo
[NeMo I 2022-04-17 10:46:36 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-17 10:46:38 mixins:167] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2022-04-17 10:46:39 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/nemo_asr_set_2.0/RES/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: true
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data2/nemo_asr_set_2.0/RES/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-04-17 10:46:39 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manifest

[NeMo I 2022-04-17 10:46:39 features:259] PADDING: 0
[NeMo I 2022-04-17 10:46:39 features:276] STFT using torch


[NeMo W 2022-04-17 10:46:39 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      "num_layers={}".format(dropout, num_layers))
    
[NeMo W 2022-04-17 10:46:40 rnnt:745] `experimental_fuse_loss_wer` will be deprecated in NeMo 1.6. Please use `fuse_loss_wer` instead.


[NeMo I 2022-04-17 10:46:40 rnnt_models:210] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0}
[NeMo I 2022-04-17 10:46:40 audio_preprocessing:496] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-04-17 10:46:44 save_restore_connector:158] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_transducer_small/a755afe69952642a8410330876938b83/stt_en_conformer_transducer_small.nemo.


In [ ]:
conformer_tr_small_output_manifest_clean="conformer_tr_small_output_manifest_clean.json"
conformer_tr_small_output_manifest_other="conformer_tr_small_output_manifest_other.json"
conformer_tr_small_model="stt_en_conformer_transducer_small" 

In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_tr_small_model \
    dataset_manifest=$test_clean_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_tr_small_output_manifest_clean


[NeMo I 2022-04-17 10:48:13 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_transducer_small
    audio_dir: null
    dataset_manifest: ./datasets/test_clean.json
    output_filename: conformer_tr_small_output_manifest_clean.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta

[NeMo W 2022-04-17 10:48:17 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/nemo_asr_set_2.0/RES/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: true
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data2/nemo_asr_set_2.0/RES/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-04-17 10:48:17 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manifest

[NeMo I 2022-04-17 10:48:17 features:259] PADDING: 0
[NeMo I 2022-04-17 10:48:17 features:276] STFT using torch


[NeMo W 2022-04-17 10:48:17 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      "num_layers={}".format(dropout, num_layers))
    
[NeMo W 2022-04-17 10:48:17 rnnt:745] `experimental_fuse_loss_wer` will be deprecated in NeMo 1.6. Please use `fuse_loss_wer` instead.


[NeMo I 2022-04-17 10:48:17 rnnt_models:210] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0}
[NeMo I 2022-04-17 10:48:17 audio_preprocessing:496] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-04-17 10:48:18 save_restore_connector:158] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_transducer_small/a755afe69952642a8410330876938b83/stt_en_conformer_transducer_small.nemo.


[NeMo W 2022-04-17 10:48:18 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-17 10:48:18 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-17 10:48:18 rnnt_wer:366] Joint fused batch size <= 0; Will temporarily disable fused batch step in the Joint.
[NeMo I 2022-04-17 10:48:18 rnnt_bpe_models:377] Changed decoding strategy to 
    strategy: greedy_batch
    compute_hypothesis_token_set: false
    preserve_alignments: null
    fused_batch_size: -1
    greedy:
      max_symbols_per_step: 10
      preserve_alignments: false
    beam:
      beam_size: 4
      search_type: default
      score_norm: true
      return_best_hypothesis: true
      tsd_max_sym_exp_per_step: 50
      alsd_max_target_len: 1.0
      nsc_max_timesteps_expansion: 1
      nsc_prefix_alpha: 1
      maes_num_steps: 2
      maes_prefix_alpha: 1
      maes_expansion_gamma: 2.3
      maes_expansion_beta: 2
      language_model: null
      softmax_temperature: 1.0
      preserve_alignments: false
    
[NeMo I 2022-04-17 10:48:18 transcribe_speech:172] 
    Transcribing 2620 files...
    
[NeMo I 2022-04-17 10:48:18 transcribe_speech:176] AMP en

Transcribing:   0%|          | 0/82 [00:00<?, ?it/s]

[NeMo I 2022-04-17 10:55:13 transcribe_speech:224] Finished transcribing 2620 files !
[NeMo I 2022-04-17 10:55:13 transcribe_speech:226] Writing transcriptions into file: conformer_tr_small_output_manifest_clean.json
[NeMo I 2022-04-17 10:55:13 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_tr_small_output_manifest_clean.json" \
    use_cer=False \
    only_score_manifest=True 

[NeMo I 2022-04-17 10:55:17 speech_to_text_eval:145] Got WER of 0.029462111990261716


In [ ]:
%run -i scripts/transcribe_speech.py \
    pretrained_name=$conformer_tr_small_model \
    dataset_manifest=$test_other_manifest \
    batch_size=32 \
    amp=True \
    output_filename=$conformer_tr_small_output_manifest_other


[NeMo I 2022-04-17 10:55:39 transcribe_speech:100] Hydra config: model_path: null
    pretrained_name: stt_en_conformer_transducer_small
    audio_dir: null
    dataset_manifest: ./datasets/test_other.json
    output_filename: conformer_tr_small_output_manifest_other.json
    batch_size: 32
    num_workers: 0
    cuda: null
    amp: true
    audio_type: wav
    overwrite_transcripts: true
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      preserve_alignments: null
      fused_batch_size: -1
      greedy:
        max_symbols_per_step: 10
        preserve_alignments: false
      beam:
        beam_size: 4
        search_type: default
        score_norm: true
        return_best_hypothesis: true
        tsd_max_sym_exp_per_step: 50
        alsd_max_target_len: 1.0
        nsc_max_timesteps_expansion: 1
        nsc_prefix_alpha: 1
        maes_num_steps: 2
        maes_prefix_alpha: 1
        maes_expansion_gamma: 2.3
        maes_expansion_beta

[NeMo W 2022-04-17 10:55:43 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/nemo_asr_set_2.0/RES/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: true
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data2/nemo_asr_set_2.0/RES/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-04-17 10:55:43 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manifest

[NeMo I 2022-04-17 10:55:43 features:259] PADDING: 0
[NeMo I 2022-04-17 10:55:43 features:276] STFT using torch


[NeMo W 2022-04-17 10:55:43 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      "num_layers={}".format(dropout, num_layers))
    
[NeMo W 2022-04-17 10:55:43 rnnt:745] `experimental_fuse_loss_wer` will be deprecated in NeMo 1.6. Please use `fuse_loss_wer` instead.


[NeMo I 2022-04-17 10:55:43 rnnt_models:210] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0}
[NeMo I 2022-04-17 10:55:43 audio_preprocessing:496] Numba CUDA SpecAugment kernel is being used
[NeMo I 2022-04-17 10:55:44 save_restore_connector:158] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_transducer_small/a755afe69952642a8410330876938b83/stt_en_conformer_transducer_small.nemo.


[NeMo W 2022-04-17 10:55:44 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
      category=PossibleUserWarning,
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2022-04-17 10:55:44 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2110: LightningDeprecationWarning: `Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.num_devices` instead.
      "`Trainer.num_gpus` was deprecated in v1.6 and will be removed in v1.8."
    


[NeMo I 2022-04-17 10:55:44 rnnt_wer:366] Joint fused batch size <= 0; Will temporarily disable fused batch step in the Joint.
[NeMo I 2022-04-17 10:55:44 rnnt_bpe_models:377] Changed decoding strategy to 
    strategy: greedy_batch
    compute_hypothesis_token_set: false
    preserve_alignments: null
    fused_batch_size: -1
    greedy:
      max_symbols_per_step: 10
      preserve_alignments: false
    beam:
      beam_size: 4
      search_type: default
      score_norm: true
      return_best_hypothesis: true
      tsd_max_sym_exp_per_step: 50
      alsd_max_target_len: 1.0
      nsc_max_timesteps_expansion: 1
      nsc_prefix_alpha: 1
      maes_num_steps: 2
      maes_prefix_alpha: 1
      maes_expansion_gamma: 2.3
      maes_expansion_beta: 2
      language_model: null
      softmax_temperature: 1.0
      preserve_alignments: false
    
[NeMo I 2022-04-17 10:55:44 transcribe_speech:172] 
    Transcribing 2939 files...
    
[NeMo I 2022-04-17 10:55:44 transcribe_speech:176] AMP en

Transcribing:   0%|          | 0/92 [00:00<?, ?it/s]

[NeMo I 2022-04-17 11:02:38 transcribe_speech:224] Finished transcribing 2939 files !
[NeMo I 2022-04-17 11:02:38 transcribe_speech:226] Writing transcriptions into file: conformer_tr_small_output_manifest_other.json
[NeMo I 2022-04-17 11:02:39 transcribe_speech:244] Finished writing predictions !


In [ ]:
%run -i scripts/speech_to_text_eval.py \
    dataset_manifest="./conformer_tr_small_output_manifest_other.json" \
    use_cer=False \
    only_score_manifest=True

[NeMo I 2022-04-17 11:02:57 speech_to_text_eval:145] Got WER of 0.06652274420648416


In [ ]:
conformer_tr_small.summarize()

[NeMo W 2022-04-17 11:03:15 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: LightningDeprecationWarning: The `LightningModule.summarize` method is deprecated in v1.5 and will be removed in v1.7. Use `pytorch_lightning.utilities.model_summary.summarize` instead.
      exec(code_obj, self.user_global_ns, self.user_ns)
    


  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoder                  | 13.0 M
2 | decoder           | RNNTDecoder                       | 1.1 M 
3 | joint             | RNNTJoint                         | 488 K 
4 | loss              | RNNTLoss                          | 0     
5 | spec_augmentation | SpectrogramAugmentation           | 0     
6 | wer               | RNNTBPEWER                        | 0     
------------------------------------------------------------------------
14.6 M    Trainable params
0         Non-trainable params
14.6 M    Total params
58.443    Total estimated model params size (MB)

## Число параметров

In [ ]:
model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_small_ls")

[NeMo I 2022-04-17 10:36:26 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_small_ls/versions/1.0.0/files/stt_en_conformer_ctc_small_ls.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo
[NeMo I 2022-04-17 10:36:30 common:705] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-17 10:36:30 mixins:167] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-04-17 10:36:30 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2022-04-17 10:36:30 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2022-04-17 10:36:30 features:259] PADDING: 0
[NeMo I 2022-04-17 10:36:30 features:276] STFT using torch
[NeMo I 2022-04-17 10:36:31 save_restore_connector:158] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0rc0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo.


In [ ]:
model.summarize()

[NeMo W 2022-04-17 10:36:35 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: LightningDeprecationWarning: The `LightningModule.summarize` method is deprecated in v1.5 and will be removed in v1.7. Use `pytorch_lightning.utilities.model_summary.summarize` instead.
      exec(code_obj, self.user_global_ns, self.user_ns)
    


  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoder                  | 13.0 M
2 | decoder           | ConvASRDecoder                    | 22.8 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WERBPE                            | 0     
------------------------------------------------------------------------
13.0 M    Trainable params
0         Non-trainable params
13.0 M    Total params
51.982    Total estimated model params size (MB)